# Import Libraries

In [ ]:
from IPython.display import display, HTML

# display(HTML("<style>.container { width:100% !important; }</style>"))

%load_ext autoreload
%autoreload 2

%load_ext jupyter_black

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import polars as pl
from pathlib import Path
from PIL import Image

# Load the dataset

In [ ]:
magazine_covers = Path.cwd().parent / "data" / "raw" / "magazine_covers.csv"
magazine_df = pl.scan_csv(magazine_covers).filter(pl.col.is_blank_cover != 1)

In [ ]:
image_folder = Path.cwd().parent / "data" / "interim" / "kmeans"
image_list = list(image_folder.glob("*.parquet"))

In [ ]:
kmeans_df = pl.scan_parquet(image_list).select(pl.exclude("og_image", "image_labels"))

In [ ]:
image_df = kmeans_df.join(other=magazine_df, on="filename", how="left").filter(
    pl.col.is_blank_cover.is_not_null()
)

In [ ]:
full_df = image_df.collect(streaming=True)
full_df

# Color Analysis (First three images)

In [ ]:
first_three_images = full_df.head(10)

In [ ]:
first_three_images

In [ ]:
for col in first_three_images.get_column("filepath"):
    im = Image.open(col)
    plt.imshow(im)
    plt.show(im)

In [ ]:
from sklearn.cluster import KMeans

with Image.open("Viceroy_Butterfly.jpg") as img:
    # Convert any RGBA -> RGB
    if np.asarray(img).shape[2] == 4:
        X: np.ndarray = np.asarray(img.convert(mode="RGB")).reshape(-1, 3)
    else:
        X = np.asarray(img).reshape(-1, 3)
    kmeans: KMeans = KMeans(n_clusters=2)
    kmeans.fit(X)
    print(kmeans.cluster_centers_[kmeans.labels_])
    # print(kmeans.labels_)

In [ ]:
np.array(
    [[173.93472002, 193.97689932, 109.05254231], [72.53669112, 73.44085545, 36.9826309]]
)[np.array([0, 1, 1, 1])]

In [ ]:
kmeans.labels_

In [ ]:
from PIL import Image, ImageDraw

# Create a new image with a white background
image = Image.new("RGB", (200, 200), "white")
draw = ImageDraw.Draw(image)

# Define the coordinates for the big square
big_square_coords = (0, 0, 200, 200)

# Define the coordinates for the three inner squares
inner_square1_coords = (25, 25, 100, 175)
inner_square2_coords = (100, 25, 175, 100)
inner_square3_coords = (100, 100, 175, 175)

# Draw the big square
draw.rectangle(big_square_coords, fill=(255, 0, 0))

# Draw the three inner squares
draw.rectangle(inner_square1_coords, fill=(0, 255, 0))
draw.rectangle(inner_square2_coords, fill=(0, 0, 255))
draw.rectangle(inner_square3_coords, fill=(255, 255, 0))

# Display the image
plt.imshow(image)
plt.axis("off")  # Turn off the axis
plt.show()
# Save the image to disk without white border
# plt.savefig(
#     "colored_squares_no_border.webp", bbox_inches="tight", pad_inches=0, dpi=300
# )